<a href="https://colab.research.google.com/github/SaraGallego22/Bankruptcy_Prediction/blob/main/Copia_de_Clsificaci%C3%B3n_Optimizacion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos dependencias.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import random
from PIL import Image
from sklearn.model_selection import  train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D

Descompresión de carpertas.

In [ ]:
!unzip enfermedades_plantas.zip

Creación de carpetas.

In [ ]:
!mkdir plantas

Se agregan las imágenes.

In [ ]:
!mv ./Tomato___Early_blight/ ./plantas



In [ ]:
!mv ./Tomato___healthy/ ./plantas

In [ ]:
import pathlib
data_dir = pathlib.Path('./plantas')

In [ ]:
dim1 = []
dim2 = []

for i in range(0,43):
    labels = str(data_dir) + '/Tomato___healthy'.format(i)
    image_path = os.listdir(labels)
    for x in image_path:
        img = imread(labels + '/' + x)
        dim1.append(img.shape[0])
        dim2.append(img.shape[1])

In [ ]:
print("Dimension 1 Mean : ",np.mean(dim1), " Dimension 2 Mean : ",np.mean(dim2))

In [ ]:
print(img)

Separamos el batch size.

In [ ]:
batch_size = 50
img_height = 256
img_width = 256

Entrenamiento del modelo.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# for element in val_ds:
#   print(element)

In [ ]:
verdaderos = np.concatenate([y for x, y in val_ds], axis=0)
verdaderos.shape

Impresión de clases.

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Modelo.

In [ ]:
num_classes = len(class_names)

model = Sequential()

model.add(layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))
model.add(layers.Conv2D(8, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(8, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(8, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='sigmoid'))
model.add(layers.Dense(num_classes))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Evaluación del modelo.

In [ ]:
epochs=50
history = model.fit(
  train_ds,
  validation_data =val_ds,
  epochs=epochs
)

Matriz de confusión

In [ ]:
# #y_pred=model.predict(val_ds)
# np.where(y_pred > 0.5, 1,0)
# y_true=model.predict(train_ds)
# np.where(y_true > 0.5, 1,0)

In [ ]:
predict_x=model.predict(val_ds)
classes_x=np.argmax(predict_x,axis=1)
classes_x

In [ ]:
trues = np.concatenate([y for x, y in val_ds], axis=0)
trues

In [ ]:

con_mat = tf.math.confusion_matrix(labels=trues, predictions=classes_x).numpy()

con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = class_names, 
                     columns = class_names)

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
print(class_names)